In [48]:
import librosa                                     
import numpy as np
import pdb
import string
from Levenshtein import distance
import glob
import numpy as np
import random
import librosa
import keras
from keras.layers import LSTM, Dense, Dropout, Flatten
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint



In [49]:
##################################################################### Utilis #################################################################
def wav2feat(wavfile):
    '''
    Input: audio wav file name
    Output: Magnitude spectrogram
    '''
    x, Fs = librosa.load(wavfile, sr=44100, mono=True) 
    hop = int(0.01 * Fs) # 10ms
    win = int(0.02 * Fs) # 20ms
    X = librosa.stft(x, n_fft=1024, hop_length=hop, win_length=win, window='hann', center=True, pad_mode='reflect')
    return np.abs(X)

def wavs2feat(wavfiles):
    '''
    Concatenate the audio files listed in wavfiles
    Input: list of audio wav file names
    Output: Magnitude spectrogram of concatenated wav
    '''
    x = []
    for wf in wavfiles:
        x1, Fs = librosa.load(wf, sr=44100, mono=True)
        x.append(x1)
    x = np.hstack(x)
    hop = int(0.01 * Fs) # 10ms
    win = int(0.02 * Fs) # 20ms
    X = librosa.stft(x, n_fft=1024, hop_length=hop, win_length=win, window='hann', center=True, pad_mode='reflect')
    return np.abs(X)

def read_csv(filename):
    id_label = {}
    with open(filename,'r') as fid:
        for line in fid: # '176787-5-0-27.wav,engine_idling\n'
            tokens = line.strip().split(',') # ['176787-5-0-27.wav', 'engine_idling']
            id_label[tokens[0]] = tokens[1]
    return id_label

def editDistance(gt, est):
    '''both are lists of labels
    E.g. gt is "dog_bark-street_music-engine_idling"
    E.g. est is "street_music-engine_idling"
    '''
    gttokens = gt.split('-')
    esttokens = est.split('-')
    # Map token to char
    tokenset = list(set(gttokens+esttokens)) # ['dog_bark', 'siren', 'street_music', 'engine_idling']
    token_char = {}
    for i in range(len(tokenset)):
        token_char[tokenset[i]] = string.ascii_uppercase[i]  # {'dog_bark': 'A', 'siren': 'B', 'street_music': 'C', 'engine_idling': 'D'}
    # convert gt and est to strings
    gtstr = [token_char[t] for t in gttokens]
    gtstr = ''.join(gtstr)  # 'BCA'
    eststr = [token_char[t] for t in esttokens]
    eststr = ''.join(eststr)  # 
    # Compare
    editdist = distance(gtstr, eststr) # 1
    score = 1 - editdist/len(gtstr)
    return editdist, score

def evals(gtcsv, estcsv, taskid):
    gt_id_label = read_csv(gtcsv)
    est_id_label = read_csv(estcsv)
    score = 0
    for id in est_id_label:
        if taskid==1:
            if est_id_label[id] == gt_id_label[id]:
                score += 1
        elif taskid==2:
            _, ss = editDistance(gt_id_label[id], est_id_label[id])
            score += ss
        else:
            pdb.set_trace()
            assert False, ["taskid not correct; it is", taskid]
    avgScore = score/len(est_id_label)
    return avgScore

#######################################################################################################################################################

In [29]:
# make list of file names
import glob          
file_names = glob.glob("../shared_train/audio_train/*.wav")

In [30]:
n= len(file_names)

In [ ]:
# do feature extraction with wav2feat and append it in a list X
X=[]
for a in file_names:
    X.append(wav2feat(a))

In [ ]:
# do padding of each vector in X

for i in range(len(X)):
    X[i]=np.pad(X[i], ((0,0),(0,401-X[i].shape[1])), 'constant')

In [6]:
# make list of all output

dic = read_csv('../shared_train/labels_train.csv')
y=[]
for key in dic:
    y.append(dic[key])

In [7]:
#  here l is dictionary which maps all labels with corresponding numbers eg- dog_bark: 0
# and Y is list of output

dic = read_csv('../shared_train/labels_train.csv')
Y=[]
l={}
i=0
for key in dic:
    i=i+1
    if i==1: continue
    Y.append(dic[key])
    l[dic[key]]=0
i=0
for key in l:
    if l[key]==0: i=i+1
    l[key]=i-1
    
y = np.zeros((len(Y),1))
for i in range(len(Y)):
    y[i] = l[Y[i]]
y=np.array(y)

In [21]:
# creating a dictionary which maps numbers to corresponding class eg- 0: dog_bark

dict_y={}
for key in l:
  dict_y[l[key]]=key
dict_y

{0: 'dog_bark',
 1: 'gun_shot',
 2: 'engine_idling',
 3: 'siren',
 4: 'jackhammer',
 5: 'drilling',
 6: 'children_playing',
 7: 'street_music',
 8: 'air_conditioner',
 9: 'car_horn'}

In [9]:
# function of converting y to one hot vector

def one_hot_encoding(y):
    
    y_1hot=np.zeros((len(y),n_classes))
    for i in range(len(y)):
        y_1hot[i,int(y[i])]=1

    return y_1hot

In [ ]:
y=one_hot_encoding(y)

In [ ]:
# spliting data into train and test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [ ]:
# defining features used to train a model

n_features=X[0].shape[0]
max_length=X[0].shape[1]

learning_rate = 0.001
batch_size = 64
n_epochs = 10
dropout = 0.3

input_shape = (n_features, max_length)
steps_per_epoch = 5

In [ ]:
# defining the model

model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=input_shape,dropout=dropout))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(n_classes, activation='softmax'))

In [ ]:
# compiling the model
# due to callback the model will be saved once it runs

opt = Adam(lr=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=opt,
metrics=['accuracy'])
callbacks = [EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20),
               ModelCheckpoint('../shared_train/_model_.h5', monitor='val_loss', mode='min', save_best_only=True)]
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 513, 256)          673792    
_________________________________________________________________
flatten_1 (Flatten)          (None, 131328)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16810112  
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 17,485,194
Trainable params: 17,485,194
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# changing input data to numpy array
X_train=np.array(X_train)
y_train=np.array(y_train)
X_test=np.array(X_test)
y_test=np.array(y_test)

In [ ]:
# fitting the model 
model.fit(X_train, y_train, batch_size=64, epochs=80, validation_data=(X_test, y_test), callbacks=callbacks )

Train on 1584 samples, validate on 177 samples
Epoch 1/80
1584/1584 [==============================] - 88s 55ms/step - loss: 2.5928 - accuracy: 0.2146 - val_loss: 1.7735 - val_accuracy: 0.2486
Epoch 2/80
1584/1584 [==============================] - 123s 78ms/step - loss: 1.7316 - accuracy: 0.3725 - val_loss: 1.4612 - val_accuracy: 0.4802
Epoch 3/80
1584/1584 [==============================] - 123s 78ms/step - loss: 1.4712 - accuracy: 0.4729 - val_loss: 1.4339 - val_accuracy: 0.4746
Epoch 4/80
1584/1584 [==============================] - 92s 58ms/step - loss: 1.2861 - accuracy: 0.5429 - val_loss: 1.1009 - val_accuracy: 0.6328
Epoch 5/80
1584/1584 [==============================] - 89s 56ms/step - loss: 1.1256 - accuracy: 0.6054 - val_loss: 1.0455 - val_accuracy: 0.6158
Epoch 6/80
1584/1584 [==============================] - 91s 57ms/step - loss: 0.9526 - accuracy: 0.6660 - val_loss: 0.9227 - val_accuracy: 0.6723
Epoch 7/80
1584/1584 [==============================] - 93s 59ms/step - los

In [61]:
model = keras.models.load_model('../shared_train/_model_.h5')

In [62]:
#################################################### testing model for sample_task_1 ###################################################################3

# creating list of file address
import glob          
input_files = glob.glob("../shared_train/feats_task_1/*.npy")
input_files.sort()
input_files

['../shared_train/feats_task_1\\a001.npy',
 '../shared_train/feats_task_1\\a002.npy',
 '../shared_train/feats_task_1\\a003.npy',
 '../shared_train/feats_task_1\\a004.npy',
 '../shared_train/feats_task_1\\a005.npy',
 '../shared_train/feats_task_1\\a006.npy',
 '../shared_train/feats_task_1\\a007.npy',
 '../shared_train/feats_task_1\\a008.npy',
 '../shared_train/feats_task_1\\a009.npy',
 '../shared_train/feats_task_1\\a010.npy',
 '../shared_train/feats_task_1\\a011.npy',
 '../shared_train/feats_task_1\\a012.npy',
 '../shared_train/feats_task_1\\a013.npy',
 '../shared_train/feats_task_1\\a014.npy',
 '../shared_train/feats_task_1\\a015.npy',
 '../shared_train/feats_task_1\\a016.npy',
 '../shared_train/feats_task_1\\a017.npy',
 '../shared_train/feats_task_1\\a018.npy',
 '../shared_train/feats_task_1\\a019.npy',
 '../shared_train/feats_task_1\\a020.npy',
 '../shared_train/feats_task_1\\a021.npy',
 '../shared_train/feats_task_1\\a022.npy',
 '../shared_train/feats_task_1\\a023.npy',
 '../shared

In [63]:
# making list of input test data and padding it

list_2=[]
for i in input_files:
    a=np.load(i)
    a=np.pad(a, ((0,0),(0,401-a.shape[1])))
    list_2.append(a)

In [64]:
# converting it to numpy array
list_2=np.array(list_2)

In [59]:
# final output of numbers
y_2=np.argmax(model.predict(list_2),axis=1)
y_2

array([8, 3, 2, 4, 0, 0, 7, 2, 2, 3, 0, 2, 3, 7, 2, 8, 6, 7, 1, 0, 8, 3,
       1, 2, 0, 4, 4, 7, 4, 6, 7, 7, 3, 6, 6, 2, 0, 1, 8, 1, 5, 4, 2, 8,
       2, 6, 6, 7, 2, 0], dtype=int64)

In [67]:
# final corresponding labels
list_2=[]
for i in range(len(y_2)):
    list_2.append(dict_y[y_2[i]])
list_2

['air_conditioner',
 'siren',
 'engine_idling',
 'jackhammer',
 'dog_bark',
 'dog_bark',
 'street_music',
 'engine_idling',
 'engine_idling',
 'siren',
 'dog_bark',
 'engine_idling',
 'siren',
 'street_music',
 'engine_idling',
 'air_conditioner',
 'children_playing',
 'street_music',
 'gun_shot',
 'dog_bark',
 'air_conditioner',
 'siren',
 'gun_shot',
 'engine_idling',
 'dog_bark',
 'jackhammer',
 'jackhammer',
 'street_music',
 'jackhammer',
 'children_playing',
 'street_music',
 'street_music',
 'siren',
 'children_playing',
 'children_playing',
 'engine_idling',
 'dog_bark',
 'gun_shot',
 'air_conditioner',
 'gun_shot',
 'drilling',
 'jackhammer',
 'engine_idling',
 'air_conditioner',
 'engine_idling',
 'children_playing',
 'children_playing',
 'street_music',
 'engine_idling',
 'dog_bark']

In [68]:
# creating list of file names
list_1=[]
for i in range(len(input_files)):
    list_1.append(input_files[i].split('\\')[-1].split('.')[0])

In [69]:
# saving result to csv file
import pandas as pd
df = pd.DataFrame(data={"col1": list_1, "col2": list_2})
df.to_csv("./final_output_task_1.csv", sep=',',index=False,header=False)
